In [2]:
from cmlextensions.ray_cluster import RayCluster

cluster = RayCluster(num_workers=2, worker_cpu=1, worker_memory=2, head_cpu=1, head_memory=1,worker_nvidia_gpu=1, dashboard_port=8080)
cluster.init()

Starting ray head...
Could not start ray head.
Could not start some of the ray workloads. Ensure ray is able to run in your environment and you have the resources in your CML workspace to provision the specified amount of ray workloads.
Set a longer timeout period if your CML workspace needs time to scale.
Shutting down Ray cluster..


In [5]:
!ray start --head

Usage stats collection is disabled.

Local node IP: 100.100.99.7
2024-08-22 10:13:52,131	WARNING services.py:2017 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=1.82gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.

--------------------
Ray runtime started.
--------------------

Next steps
  To add another node to this Ray cluster, run
    ray start --address='100.100.99.7:6379'
  
  To connect to this Ray cluster:
    import ray
    ray.init()
  
  To terminate the Ray runtime, run
    ray stop
  
  To view the status of the cluster, use
    ray status


In [7]:
!ray dashboard -p $CDSW_READONLY_PORT 

Usage: ray dashboard [OPTIONS] CLUSTER_CONFIG_FILE
Try 'ray dashboard --help' for help.

Error: Missing argument 'CLUSTER_CONFIG_FILE'.


In [3]:
!pip install git+https://github.com/cloudera/cmlextensions.git

  Cloning https://github.com/cloudera/cmlextensions.git to /tmp/pip-req-build-28n__tyn
  Running command git clone --filter=blob:none --quiet https://github.com/cloudera/cmlextensions.git /tmp/pip-req-build-28n__tyn
  Resolved https://github.com/cloudera/cmlextensions.git to commit d30304e29ab636d238b612119c54648825fb149d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cmlextensions: filename=cmlextensions-0.1-py3-none-any.whl size=13977 sha256=7006a0ae9d082cdf430dea2b845fb98d9dc17a908a21e148b79838896740d035
  Stored in directory: /tmp/pip-ephem-wheel-cache-cpg5l01f/wheels/7f/6d/42/9bc19a827652fdc63a1e3379c87895185da77124f921b4fbdb
Successfully built cmlextensions


In [5]:
from cmlextensions.ray_cluster import RayCluster

In [6]:

import ray
import modin.pandas as pd
import numpy as np

In [8]:
# Initialize Ray with GPU resources
ray.init()

@ray.remote(num_gpus=0.5)
def process_large_dataset(task_id, num_rows):
    print(f"Task {task_id} using 0.5 GPU started.")
    
    # Create a large DataFrame using Modin
    df = pd.DataFrame(np.random.rand(num_rows, 10), columns=[f'col_{i}' for i in range(10)])
    
    # Perform some operations on the DataFrame
    result = df.apply(lambda x: x ** 2).sum()
    
    print(f"Task {task_id} using 0.5 GPU completed.")
    return f"Result of task {task_id} is:\n{result}"

@ray.remote(num_gpus=0.25)
def filter_large_dataset(task_id, num_rows):
    print(f"Task {task_id} using 0.25 GPU started.")
    
    # Create a large DataFrame using Modin
    df = pd.DataFrame(np.random.rand(num_rows, 10), columns=[f'col_{i}' for i in range(10)])
    
    # Filter the DataFrame
    filtered_df = df[df['col_0'] > 0.5]
    
    print(f"Task {task_id} using 0.25 GPU completed.")
    return f"Filtered DataFrame for task {task_id} has {len(filtered_df)} rows"

In [9]:
def main():
    num_rows = 10**6  # Example size of the dataset

    # Launch tasks with GPU slicing
    task1 = process_large_dataset.remote(1, num_rows)
    task2 = filter_large_dataset.remote(2, num_rows)
    task3 = process_large_dataset.remote(3, num_rows)

    # Gather results
    results = ray.get([task1, task2, task3])

    # Print results
    for result in results:
        print(result)

if __name__ == "__main__":
    main()
    ray.shutdown()



(process_large_dataset pid=7306) Task 1 using 0.5 GPU started.


(process_large_dataset pid=7306) UserWarning: Distributing <class 'numpy.ndarray'> object. This may take some time.


(filter_large_dataset pid=7308) Task 2 using 0.25 GPU completed.
(filter_large_dataset pid=7308) Task 2 using 0.25 GPU started.
(process_large_dataset pid=8532) Task 3 using 0.5 GPU started.
(process_large_dataset pid=7306) Task 1 using 0.5 GPU completed.


(process_large_dataset pid=8532) UserWarning: Distributing <class 'numpy.ndarray'> object. This may take some time. [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


Result of task 1 is:
col_0    333505.092446
col_1    333221.526196
col_2    333303.259179
col_3    333653.284097
col_4    333705.498627
col_5    333112.887854
col_6    332928.717787
col_7    332668.340261
col_8    333627.419843
col_9    333288.985124
dtype: float64
Filtered DataFrame for task 2 has 499360 rows
Result of task 3 is:
col_0    333202.479034
col_1    333062.903244
col_2    333455.487117
col_3    333298.166438
col_4    333654.189940
col_5    333604.595087
col_6    333584.404244
col_7    333362.632928
col_8    333527.654247
col_9    333377.207994
dtype: float64
